# Introduction
This notebook shows how to use the [Microsoft Graph Security API](https://aka.ms/graphsecuritydocs). It defines a Python wrapper for calling the API to make it easy to access [Microsoft Graph Security alerts](https://aka.ms/graphsecurityalerts). While the APIs are documented and supported,
the wrapper in this notebook is a sample and is not an official programming interface.

Author: [Microsoft Graph Security Team](graphsecfeedback@microsoft.com) (Inspired by [@JohnLaTwC](https://twitter.com/JohnLaTwC))

## Links to Microsoft Graph Security API documentation and other samples

[Overview of Microsoft Graph Security](https://aka.ms/graphsecuritydocs)

[What are alerts?](https://aka.ms/graphsecurityalerts)

[Alert Schema - list of properties and descriptions](https://docs.microsoft.com/en-us/graph/api/resources/alert?view=graph-rest-1.0) 

[Authorization - Application-only and User-Delegated](https://docs.microsoft.com/graph/security-authorization?view=graph-rest-1.0)

[More Samples](https://aka.ms/graphsecurityapicode)

[OData query examples](https://docs.microsoft.com/graph/query-parameters)

# Authentication - Initialize secrets

Download this notebook and follow the steps below to get the application ID and secret that you can use to authenticate and get data from your Azure Active Directory (Azure AD) tenant using this notebook. 
1.	[Register your application](https://docs.microsoft.com/en-us/graph/auth-v2-service#1-register-your-app) for this notebook in Azure AD in application only mode. 
2.	[Configure permissions](https://docs.microsoft.com/en-us/graph/auth-v2-service#2-configure-permissions-for-microsoft-graph) and be sure to add the `SecurityEvents.ReadWrite.All` permission to your application.
3. Get your Azure AD tenant administrator to [grant tenant administration consent](https://docs.microsoft.com/en-us/graph/auth-v2-service#3-get-administrator-consent) to your application. This is a one-time activity unless permissions change for the application. 

When your app is registered to call the Microsoft Graph Security API you need to pass the application ID and application secret from the above mentioned steps in to this sample. 

Use either plain text input to enter your secret in the `get_secret` function or get it from your environment variable in the `get_secret` function below the first one. Use only one of these functions to get secret. 

In [7]:
def get_secret(secret_name):
    return {'GRAPHSEC_DEMO_appId':'<<Enter your registered application ID>>',
            'GRAPHSEC_DEMO_appSecret':'<<Enter you application secret>>',
            'GRAPHSEC_DEMO_tenantId':'<<Enter your tenant ID>>'}[secret_name]

In [ ]:
## e.g. custom way to access secrets
def get_secret(secret_name):
    import os
    return os.environ.get(secret_name)

In [8]:
appId = get_secret('GRAPHSEC_DEMO_appId')
appSecret = get_secret('GRAPHSEC_DEMO_appSecret') 
tenantId = get_secret('GRAPHSEC_DEMO_tenantId')
print("If your secrets were initialized properly you should see your tenant id.\nTenant Id: %s " % tenantId)

If your secrets were initialized properly you should see your tenant id.
Tenant Id: <<Enter your tenant ID>> 


# Getting Started
All the API Wrapper code is below. Activate it by clicking in the cell and hitting Shift+Enter

In [3]:
# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. 
# --------------------------------------------------------------------------

## https://aka.ms/graphsecuritydocs

import json
import urllib.request
import urllib.parse
import pandas as pd
import re
    
class MicrosoftGraphSecurityAPI:
    def __init__(self, tenantId, appId, appSecret, fUsePandas=True, 
                 api_root="https://graph.microsoft.com/", api_version="v1.0"):
        url = "https://login.microsoftonline.com/%s/oauth2/v2.0/token" % (tenantId)

        self.fDebug = False

        body = {
            'client_id' : appId,
            'client_secret' : appSecret,
            'grant_type' : 'client_credentials',
            'scope': 'https://graph.microsoft.com/.default'
        }

        ## authenticate and obtain AAD Token for future calls
        data = urllib.parse.urlencode(body).encode("utf-8")
        req = urllib.request.Request(url, data)
        response = urllib.request.urlopen(req)
        jsonResponse = json.loads(response.read().decode())
        self.aadToken = jsonResponse["access_token"]
        self.headers = { 
            'Content-Type' : 'application/json',
            'Accept' : 'application/json',
            'Authorization' : "Bearer " + self.aadToken
        }
        self.fUsePandas = fUsePandas # use pandas DataFrame for collections of objects, else return a list
        self.api_root = api_root + api_version 
        if len(self.aadToken) > 0:
            print("Connected.")
    
    def set_output_type(self, fUsePandas=True):
        self.fUsePandas = fUsePandas    

    def set_debug_output(self, fDebug=True):
        self.fDebug = fDebug    
        
    def __validate_arguments(self,args, valid_params):
        if len(args) == 0:
            raise ValueError ('argument must be one of %s' % str(list(valid_params.keys())))
        elif len(args) > 1:
            raise ValueError ('only one id can be used at a time')
        else:
            selector = next(iter(args))
            selector_value= next(iter(args.values()))
            if selector not in list(valid_params.keys()):
                raise ValueError ('argument must be one of %s' % str(list(valid_params.keys())))
        return (selector, selector_value)

    def __make_request(self,url, params=None):
 
        if self.fDebug:
            print(url)
        req = urllib.request.Request(url, headers=self.headers)
        try:
            response = urllib.request.urlopen(req)
        except urllib.error.HTTPError as e:
            raise e
                
        jsonResponse = json.loads(response.read().decode())
        if type(jsonResponse) == int:
            if self.fUsePandas:
                return pd.DataFrame([jsonResponse])
            else:
                return jsonResponse
        if 'value' in jsonResponse:
            res = jsonResponse["value"]
            if len(res) == 0:
                res = {}
        else:
            res = jsonResponse     
        if self.fUsePandas:
            return pd.io.json.json_normalize(res)
        return res

    def __prepare_param_dict_from_filter_str(self, filterstr):
        get_params = {}
        for filter_param in re.split("[\?\&]+", filterstr):
            if len(filter_param)> 0:
                attr = filter_param.split('=')[0]
                val  = filter_param.split('=')[1]
                get_params[attr]= val
        return get_params
    
    def alerts(self, **kwargs):
        alert_url = self.api_root + "/security/alerts"
        get_params = None
        
        valid_params = {
            'filterstr' : alert_url + '?%s',
            'alertid'   : alert_url + '/%s',
            'userid'    : alert_url + "?$filter=userStates/any(d:d/userPrincipalName eq '%s')",
            'ip'        : alert_url + "?$filter=hostStates/any(d:d/privateIpAddress eq '%s')",
            'hostfqdn'  : alert_url + "?$filter=hostStates/any(d:d/fqdn eq '%s')", 
            'filehash'  : alert_url + "?$filter=fileStates/any(d:d/fileHash/hashValue eq '%s')",
            'filename'  : alert_url + "?$filter=fileStates/any(d:d/name eq '%s')", 
            'domain'    : alert_url + "?$filter=networkConnections/any(d:d/destinationDomain eq '%s')" 
        }
        (selector, selector_value) = self.__validate_arguments(kwargs, valid_params)
        
        if selector == 'filterstr':
            get_params = self.__prepare_param_dict_from_filter_str(selector_value)
            if get_params is not None:
                url = valid_params[selector] % urllib.parse.urlencode(get_params)
        else:
            url = valid_params[selector] % selector_value
            url = urllib.parse.quote( url , safe="%/:=&?~#+!$,;'@()*[]") # Url encode spaces 

        return self.__make_request(url)
    
print("Sample Microsoft Graph Security API code loaded")

Sample Microsoft Graph Security API code loaded


In [5]:
# hit Shift-Enter in this cell

# NOTE: endpoint authorization will periodically time out and you will need to re-run this command to re-authenticate
# if you're able to call the API (client app registered properly), and you get an exception 
# with a HTTP error 401 Unauthorized, re-run this command to re-initiate the endpoint

MsftGraphSec_api = MicrosoftGraphSecurityAPI(tenantId, appId, appSecret)
MsftGraphSec_api.set_debug_output()

Connected.


# Testing Get Alerts

Let's query the most recent 5 alerts using the OData TOP keyword. The `$top=5` query will return 5 of the most recent alerts from *each* [Microsoft Graph Security alert provider](https://aka.ms/graphsecurityalerts). 

In [6]:
MsftGraphSec_api.alerts(filterstr = "$top=5")

https://graph.microsoft.com/v1.0/security/alerts?%24top=5


,activityGroupName,assignedTo,azureSubscriptionId,azureTenantId,category,closedDateTime,cloudAppStates,comments,confidence,createdDateTime,...,status,tags,title,triggers,userStates,vendorInformation.provider,vendorInformation.providerVersion,vendorInformation.subProvider,vendorInformation.vendor,vulnerabilityStates
0,None,,,63d0b44f-8610-4dd2-91fd-541ffc973399,threat,None,[],[],NaN,2019-04-20T15:11:31Z,...,newAlert,[],Traps: Malware Blocked,[],"[{'aadUserId': None, 'accountName': 'adamw', '...",Palo Alto Networks,5.0,Traps,Palo Alto Networks,[]
1,None,,,63d0b44f-8610-4dd2-91fd-541ffc973399,vulnerability,None,"[{'destinationServiceIp': None, 'destinationSe...",[],NaN,2019-04-20T15:58:31Z,...,newAlert,[endpoint],threat: vulnerability 35993,"[{'name': 'networkConnection.sourceAddresss', ...","[{'aadUserId': None, 'accountName': 'christiec...",Palo Alto Networks,8.1,NGFW,Palo Alto Networks,[]
2,None,,,63d0b44f-8610-4dd2-91fd-541ffc973399,threat,None,[],[],NaN,2019-04-20T15:33:15Z,...,newAlert,[],Traps: Malware Blocked,[],"[{'aadUserId': None, 'accountName': 'aldom', '...",Palo Alto Networks,5.0,Traps,Palo Alto Networks,[]
3,None,,,63d0b44f-8610-4dd2-91fd-541ffc973399,data,None,"[{'destinationServiceIp': None, 'destinationSe...",[],NaN,2019-04-17T10:16:36Z,...,newAlert,[],Data Filtering Alert,[],"[{'aadUserId': None, 'accountName': None, 'dom...",Palo Alto Networks,None,NG Firewall,Palo Alto Networks,[]
4,None,None,,63d0b44f-8610-4dd2-91fd-541ffc973399,data,None,"[{'destinationServiceIp': None, 'destinationSe...",[],NaN,2019-04-16T10:16:31Z,...,newAlert,[],Data Filtering Alert,[],"[{'aadUserId': None, 'accountName': 'douglasf'...",Palo Alto Networks,8.1,NGFW,Palo Alto Networks,[]
5,None,None,None,63d0b44f-8610-4dd2-91fd-541ffc973399,impossibleLoginVelocity,None,[],[],0.0,2019-04-17T18:30:45Z,...,newAlert,[],Impossible travel activity,[],"[{'aadUserId': None, 'accountName': 'douglasf'...",Cloud Application Security,3.0,,Microsoft,[]
6,,,None,63d0b44f-8610-4dd2-91fd-541ffc973399,repeatedShareActivity,None,[],[],0.0,2019-04-17T14:33:00Z,...,newAlert,[],Mass share,[],"[{'aadUserId': None, 'accountName': 'douglasf'...",Cloud Application Security,3.0,,Microsoft,[]
7,None,None,None,63d0b44f-8610-4dd2-91fd-541ffc973399,repeatedActivityShare,None,[],[],0.0,2019-04-16T16:55:36Z,...,newAlert,[],Mass share,[],"[{'aadUserId': None, 'accountName': 'douglasf'...",Cloud Application Security,3.0,,Microsoft,[]
8,None,None,None,63d0b44f-8610-4dd2-91fd-541ffc973399,repeatedShareActivity,None,[],[],0.0,2019-04-15T21:22:00Z,...,newAlert,[],Mass share,[],"[{'aadUserId': None, 'accountName': 'aldom', '...",Cloud Application Security,3.0,,Microsoft,[]
9,None,None,None,63d0b44f-8610-4dd2-91fd-541ffc973399,repeatedActivityShare,None,[],[],0.0,2019-04-15T17:10:00Z,...,newAlert,[],Mass share,[],"[{'aadUserId': None, 'accountName': 'pattif', ...",Cloud Application Security,3.0,,Microsoft,[]


# Security Investigation Scenarios
You can [query alerts](https://docs.microsoft.com/en-us/graph/api/alert-list?view=graph-rest-1.0) with OData queries. OData provides a simple standardized syntax for selecting properties and filtering data

A summary of the OData filters can be found [here](https://docs.microsoft.com/graph/query-parameters) <https://docs.microsoft.com/graph/query-parameters>

## Querying alerts by UPN (User Principal Name)

In [ ]:
MsftGraphSec_api.alerts( userid = '<<Enter user principal name>>')

## Querying alerts by machine FQDNs (Fully Qualified Domain Name)
You can query alerts by machine fqdn, user id, ip, alert id, file sha1, domain, OData query, and more

In [ ]:
MsftGraphSec_api.alerts( hostfqdn = '<<Enter host fqdn>>')

In [ ]:
# query machines associated with an alert
MsftGraphSec_api.alerts( alertid = '<<Enter alert ID>>')[['id', 'hostStates']]    #[['id', 'osBuild','osPlatform']]

In [ ]:
# query alerts by machine name
MsftGraphSec_api.alerts( filterstr = "$filter=hostStates/any(d:d/netBiosName eq '<<Enter host name>>')")

## Querying alerts by private IP
You can query alerts by private IPs populated by the providers within the alert hostStates.

In [ ]:
MsftGraphSec_api.alerts( ip = '<<Enter ip address>>' )